In [3]:
import pandas as pd
import numpy as np


In [14]:
df_population = pd.read_csv('./data/population_data_byYear.csv')
df_2020 = pd.read_csv('./data/COVID data_2020.csv')
df_2021 = pd.read_csv('./data/COVID data_2021.csv')
df_2022 = pd.read_csv('./data/COVID data_2022.csv')

In [15]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Series Name    266 non-null    object
 1   Country Name   266 non-null    object
 2   Country Code   266 non-null    object
 3   2019 [YR2019]  266 non-null    object
 4   2020 [YR2020]  266 non-null    object
 5   2021 [YR2021]  266 non-null    object
dtypes: object(6)
memory usage: 12.6+ KB


In [28]:
list(df_population.columns)

['Series Name',
 'Country Name',
 'Country Code',
 '2019 [YR2019]',
 '2020 [YR2020]',
 '2021 [YR2021]']

In [35]:
df_pop = df_population[['2019 [YR2019]','2020 [YR2020]','2021 [YR2021]']].apply(
    lambda x: x.apply(
        lambda x: float(x) if x.isnumeric() else np.nan
    )
)
df_pop['Country Name'] = df_population['Country Name']
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   2019 [YR2019]  264 non-null    float64
 1   2020 [YR2020]  264 non-null    float64
 2   2021 [YR2021]  264 non-null    float64
 3   Country Name   266 non-null    object 
dtypes: float64(3), object(1)
memory usage: 8.4+ KB


In [67]:
pop_country = set(sorted(df_pop['Country Name'].unique()))

In [36]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Climate               276 non-null    object 
 1   Country Region        276 non-null    object 
 2   Country/Region        276 non-null    object 
 3   Province/State        89 non-null     object 
 4   Avg. Confirmed Cases  276 non-null    float64
dtypes: float64(1), object(4)
memory usage: 10.9+ KB


In [68]:
df_country = set(sorted(df_2020['Country Region'].unique()))

In [64]:
sum([True if p==d else False for p, d in zip(pop_country, df_country)])

1

In [70]:
len(df_country.intersection(pop_country))

166

In [37]:
def clean_df(df):
    df_c = df.drop(columns=['Country/Region', 'Province/State']).dropna()
    return df_c

In [38]:
df_2020_comp = clean_df(df_2020)
df_2022_comp = clean_df(df_2022)

In [39]:
df_2021_comp = df_2021.drop(columns=['Country/Region', 'Province/State', 'Avg. People fully vaccinated']).dropna()

In [93]:
def Merge(df, df_pop, col_year):
    df_new =pd.merge(df, 
         df_pop[['Country Name', col_year]], 
         how='outer', 
         left_on='Country Region', 
         right_on='Country Name')
    df_new.dropna(inplace=True)
    df_new = df_new.groupby(['Climate','Country Region']).mean()
    return df_new


In [96]:
df_pop_covid_2020 = Merge(df_2020_comp, df_pop, '2019 [YR2019]')
df_pop_covid_2021 = Merge(df_2021_comp, df_pop, '2020 [YR2020]')
df_pop_covid_2022 = Merge(df_2022_comp, df_pop, '2021 [YR2021]')

In [97]:
df_pop_covid_2020.to_csv('./data/population_covid_2020.csv')
df_pop_covid_2021.to_csv('./data/population_covid_2021.csv')
df_pop_covid_2022.to_csv('./data/population_covid_2022.csv')

#### Testing Merge with population dataset

In [71]:
df_test_2020 = pd.merge(df_2020_comp, 
         df_pop[['Country Name', '2019 [YR2019]']], 
         how='inner', 
         left_on='Country Region', 
         right_on='Country Name')

In [85]:
df_test_2020.shape

(252, 5)

In [73]:
display(df_test_2020.tail(10))
df_test_2020.head(10)

,Climate,Country Region,Avg. Confirmed Cases,Country Name,2019 [YR2019]
242,Subtropics,Australia,90.19,Australia,25365745.0
243,Temperate,Australia,178.13,Australia,25365745.0
244,Temperate,Armenia,42182.83,Armenia,2957728.0
245,Subtropics,Argentina,429960.64,Argentina,44938712.0
246,Tropical,Antigua and Barbuda,66.38,Antigua and Barbuda,97115.0
247,Tropical,Angola,3893.94,Angola,31825299.0
248,Temperate,Andorra,1979.19,Andorra,77146.0
249,Subtropics,Algeria,30676.45,Algeria,43053054.0
250,Temperate,Albania,10804.48,Albania,2854191.0
251,Subtropics,Afghanistan,24642.76,Afghanistan,38041757.0


,Climate,Country Region,Avg. Confirmed Cases,Country Name,2019 [YR2019]
0,Tropical,Zimbabwe,3873.28,Zimbabwe,14645473.0
1,Tropical,Zambia,7054.73,Zambia,17861034.0
2,Subtropics,West Bank and Gaza,25371.59,West Bank and Gaza,4685306.0
3,Tropical,Vietnam,635.76,Vietnam,96462108.0
4,Tropical,Vanuatu,0.15,Vanuatu,299882.0
5,Temperate,Uzbekistan,28069.10,Uzbekistan,33580350.0
6,Subtropics,Uruguay,2208.85,Uruguay,3461731.0
7,Tropical,United Kingdom,289.13,United Kingdom,66836327.0
8,Tropical,United Kingdom,0.84,United Kingdom,66836327.0
9,Tropical,United Kingdom,9.70,United Kingdom,66836327.0


In [84]:
df_test_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Climate               252 non-null    object 
 1   Country Region        252 non-null    object 
 2   Avg. Confirmed Cases  252 non-null    float64
 3   Country Name          252 non-null    object 
 4   2019 [YR2019]         252 non-null    float64
dtypes: float64(2), object(3)
memory usage: 11.8+ KB


In [79]:
df_test_2020.groupby(['Climate','Country Region']).mean()

Avg. Confirmed Cases  2019 [YR2019]
Climate  Country Region                                     
Polar    Canada                     16.413333     37601230.0
         Denmark                   143.710000      5814422.0
         Finland                  9558.020000      5521606.0
         Iceland                  2341.620000       360563.0
         Norway                  12838.780000      5347896.0
...                                       ...            ...
Tropical United Kingdom             81.166667     66836327.0
         Vanuatu                     0.150000       299882.0
         Vietnam                   635.760000     96462108.0
         Zambia                   7054.730000     17861034.0
         Zimbabwe                 3873.280000     14645473.0

[178 rows x 2 columns]